# Final Exam

### Part 0 - Data Collection

Collecting the data for Plant City, Florida (station ID = USC00087205). I'm using the in-class exercise from Module 3 as a guide to collect the data.

In [5]:
import pandas as pd
import numpy as np
import fsspec

# Setting the storage options, like in Module 3
STOR = {"anon": True}

# Station ID for Plant City Florida
station_id = 'USC00087205'

# Loading the daily data for the station ID
url = f"s3://noaa-ghcn-pds/csv/by_station/{station_id}.csv"
import_df = pd.read_csv(url, storage_options=STOR, dtype={'ID': str, 'ELEMENT': str}, parse_dates=['DATE'])

# Quick check - looks good
# df

# Making a csv I can use in the next parts
import_df.to_csv("Data/Plant_City.csv")

C:\Users\macdonaldc\AppData\Local\Temp\ipykernel_21824\3256992655.py:13: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, storage_options=STOR, dtype={'ID': str, 'ELEMENT': str}, parse_dates=['DATE'])


### Part 1 - Frost and Freeze Days

Strawberries are planted around October 1 and ready for harvest by the end of January. What is the mean risk of frost and freeze, defined as the mean number of days per month over the period 1991-2020 that the temperature has been observed to be less than or equal to 32 and 28 degrees Fahrenheit, respectively, that might damage the plants for each month during the October - January period? (25 points)

In [18]:
# Reading in the 
df = pd.read_csv("Data/Plant_City.csv", parse_dates=['DATE'])
# Quick check - still looking good
# df

# Pivoting to wide to get temperature columns
wide = df.pivot_table(index=['ID', 'DATE'], columns='ELEMENT', values='DATA_VALUE', aggfunc='first').reset_index()

# Converting the units from tenths of degrees C to degrees C.
# I think I'm only using TMIN here if I'm looking for the coldest part of the day, so not going to do the same for TMAX
if 'TMIN' in wide.columns:
    wide['TMIN'] = wide['TMIN'] / 10.0

# Units look right for TMIN
wide.head()

C:\Users\macdonaldc\AppData\Local\Temp\ipykernel_21824\2998889889.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/Plant_City.csv", parse_dates=['DATE'])


ELEMENT,ID,DATE,DAPR,MDPR,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT06,WT08,WT11,WT14,WT16
0,USC00087205,1892-09-01,NaN,NaN,NaN,NaN,NaN,322.0,20.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00087205,1892-09-02,NaN,NaN,NaN,NaN,NaN,317.0,20.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00087205,1892-09-03,NaN,NaN,NaN,NaN,NaN,317.0,21.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USC00087205,1892-09-04,NaN,NaN,NaN,NaN,NaN,322.0,21.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USC00087205,1892-09-05,NaN,NaN,NaN,NaN,NaN,333.0,21.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Looks good so far. Max TMIN is +32 and min TMIN is -9.4, which seem reasonable. 
# Looking at the 25th% percentile TMINs, it's clear frost/freeze days are going to be a very small portion of the dataset if
# 25th% is 12.2C
wide.describe()

ELEMENT,DATE,DAPR,MDPR,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT06,WT08,WT11,WT14,WT16
count,46166,13.000000,16.000000,45572.000000,24059.0,23938.0,45430.000000,45420.000000,39673.000000,7.0,51.0,1.0,1.0,1.0,44.0,16.0,5.0
mean,1960-11-15 02:24:51.279296416,2.153846,341.500000,38.098745,0.0,0.0,287.088664,16.146438,244.972677,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
min,1892-09-01 00:00:00,2.000000,3.000000,0.000000,0.0,0.0,-572.000000,-9.400000,-133.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1929-10-13 06:00:00,2.000000,27.000000,0.000000,0.0,0.0,256.000000,12.200000,211.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1961-07-05 12:00:00,2.000000,181.500000,0.000000,0.0,0.0,294.000000,17.800000,256.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1994-01-08 18:00:00,2.000000,406.750000,10.000000,0.0,0.0,328.000000,21.100000,289.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,2025-12-14 00:00:00,3.000000,1842.000000,2464.000000,0.0,0.0,400.000000,32.200000,389.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,NaN,0.375534,484.811991,109.782656,0.0,0.0,50.059821,6.265635,59.962690,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0


In [ ]:
# Convert TMIN from C to F (goobye intuitive units)
wide['TMIN_F'] = (wide['TMIN'] * 9/5) + 32

# Adding in the time columns to make filtering easier in the next steps
wide['year'] = wide['DATE'].dt.year
wide['month'] = wide['DATE'].dt.month

# Dropping rows with NaN TMIN data
wide = wide.dropna(subset=['TMIN'])

# Another check - still looking good (keeping the C TMIN in for my sake)
wide[['DATE', 'year', 'month', 'TMIN', 'TMIN_F']].head()


ELEMENT,DATE,year,month,TMIN,TMIN_F
0,1892-09-01,1892,9,20.6,69.08
1,1892-09-02,1892,9,20.6,69.08
2,1892-09-03,1892,9,21.1,69.98
3,1892-09-04,1892,9,21.7,71.06
4,1892-09-05,1892,9,21.1,69.98


In [25]:
# Checking the length before filtering
len(wide)

45420

In [24]:
# Filtering to the period 1991-2020
climo = wide[(wide['year'] >= 1991) & (wide['year'] <= 2020)].copy()

# Filtering just the growing season months (Oct - Jan)
climo = climo[climo['month'].isin([10, 11, 12, 1])]

# Checking the length after filtering. Looks about right - ~120 days per season * 30 yrs = 3600
# Looks like not much missing data over the time period
len(climo)

3617

In [27]:
# Creating the criteria for frost (<=32F) and freeze (<=28F) days
climo['frost'] = (climo['TMIN_F'] <=32)
climo['freeze'] = (climo['TMIN_F'] <= 28)

# Counting frost and freeze days per month for each year using groupby
monthly_counts = climo.groupby(['year', 'month']).agg(
    frost_days=('frost', 'sum'),
    freeze_days=('freeze', 'sum')
).reset_index()

# Another check - looking good so far, a few frost days in 1992, first freeze day is 1995
monthly_counts

,year,month,frost_days,freeze_days
0,1991,1,0,0
1,1991,10,0,0
2,1991,11,0,0
3,1991,12,0,0
4,1992,1,3,0
...,...,...,...,...
115,2019,12,0,0
116,2020,1,1,0
117,2020,10,0,0
118,2020,11,0,0


In [28]:
# Calculating the mean number of frost and freeze days per month over the entire period 1991-2020
mean_frost_freeze = monthly_counts.groupby('month').agg(
    mean_frost_days=('frost_days', 'mean'),
    mean_freeze_days=('freeze_days', 'mean')
)

mean_frost_freeze


,mean_frost_days,mean_freeze_days
month,,
1,1.866667,0.500000
10,0.000000,0.000000
11,0.033333,0.000000
12,0.600000,0.166667


The mean number of frost days is 0 days/month in Oct, 0.03 in Nov, 0.6 in Dec and 1.9 in Jan. For Freeze days, it's 0 in Oct and Nov, 0.17 days/month in Dec and 0.5 in Jan.

### Part 2 - Relating cold events to the El Nino Southern Oscillation (ENSO)
To begin to explore the seasonal to sub-seasonal prediction of freeze events at this site, using code you adapt from Module 4, we're going to try to relate these cold events to the El Nino Southern Oscillation (ENSO).  You have a hypothesis that ENSO is related to seasonal prediction of freeze events, but you don't know which region to choose for calculating your anomalies.  The problem is that there are many ENSO indicies that represent forcing across the eastern and central Pacific: which SST forcing region is most related to cold conditions in central Florida?

In [35]:
# USing data for the growing season from the entire dataset 
growing_season = wide[wide['month'].isin([10, 11, 12, 1])].copy()

# Creating the criteria for freeze days, same as above
growing_season['freeze'] = (growing_season['TMIN_F'] <= 28)

# Counting freeze days per month, same as above
freeze_monthly = growing_season.groupby(['year', 'month']).agg(
    freeze_days=('freeze', 'sum')
).reset_index()

# Creating a date index from the year/month (and setting day to first of month) so I can work with the data as datetimes
freeze_monthly['DATE'] = pd.to_datetime(
    freeze_monthly['year'].astype(str) + '-' + freeze_monthly['month'].astype(str) + '-01'
)

freeze_monthly = freeze_monthly.set_index('DATE')

# Looks good
freeze_monthly.head()

,year,month,freeze_days
DATE,,,
1892-10-01,1892,10,0
1892-11-01,1892,11,0
1892-12-01,1892,12,0
1893-01-01,1893,1,2
1893-10-01,1893,10,0


In [44]:
# Loading the ENSO indices from CPC
enso_url = 'https://www.cpc.ncep.noaa.gov/data/indices/sstoi.indices'

# Read the file, using fsspec like in Module 4 check-in
text = fsspec.open(enso_url, 'r').open().read()
lines = text.strip().split('\n')

# Printing the first few lines - looking good so far, looks just like the link
for i in range(0, 5):
    print(lines[i])

YR MON  NINO1+2   ANOM   NINO3    ANOM   NINO4    ANOM NINO3.4    ANOM
1982   1   24.28   -0.24   25.84    0.17   28.01   -0.21   26.65    0.08
1982   2   25.38   -0.72   26.26   -0.11   27.99   -0.11   26.54   -0.20
1982   3   25.22   -1.38   26.92   -0.25   28.18   -0.05   27.09   -0.14
1982   4   24.57   -1.16   27.52   -0.05   28.61    0.10   27.83    0.02


In [ ]:
# Converting the data to a datafram with the index anomalies
# Modified code form M04 check-in part 3
data = []
for line in lines:
    parts = line.split()
    if not parts or not parts[0].isdigit():
        continue
    
    year = int(parts[0])
    month = int(parts[1])
    
    nino12_anom = float(parts[3])
    nino3_anom = float(parts[5])
    nino4_anom = float(parts[7])
    nino34_anom = float(parts[9])
    
    data.append({
        'year': year,
        'month': month,
        'NINO1+2 Anom': nino12_anom,
        'NINO3 Anom': nino3_anom,
        'NINO4 Anom': nino4_anom,
        'NINO3.4 Anom': nino34_anom
    })

enso = pd.DataFrame(data)

# Creating a date index, like avove
enso['date'] = pd.to_datetime(
    enso['year'].astype(str) + '-' + enso['month'].astype(str) + '-01'
)
enso = enso.set_index('date')

# Checks - looks good. Anoms match the data in the link, datetime works
enso.head()

,year,month,NINO1+2 Anom,NINO3 Anom,NINO4 Anom,NINO3.4 Anom
date,,,,,,
1982-01-01,1982,1,-0.24,0.17,-0.21,0.08
1982-02-01,1982,2,-0.72,-0.11,-0.11,-0.20
1982-03-01,1982,3,-1.38,-0.25,-0.05,-0.14
1982-04-01,1982,4,-1.16,-0.05,0.10,0.02
1982-05-01,1982,5,-0.62,0.49,0.40,0.49


In [47]:
# Merge the datasets, using an inner join
merged = freeze_monthly.merge(enso[['NINO1+2 Anom', 'NINO3 Anom', 'NINO4 Anom', 'NINO3.4 Anom']], 
                               left_index=True, right_index=True, how='inner')

#  Spot checked a few months, looks good
merged.head()

,year,month,freeze_days,NINO1+2 Anom,NINO3 Anom,NINO4 Anom,NINO3.4 Anom
1982-01-01,1982,1,1,-0.24,0.17,-0.21,0.08
1982-10-01,1982,10,0,1.80,1.96,0.43,1.73
1982-11-01,1982,11,0,2.78,2.19,0.18,1.68
1982-12-01,1982,12,0,2.89,2.80,0.36,2.21
1983-01-01,1983,1,0,2.30,2.69,0.41,2.13


In [50]:
nino_cols = ['NINO1+2 Anom', 'NINO3 Anom', 'NINO4 Anom', 'NINO3.4 Anom']

# correlations of freeze_days vs each NINO column
correlations = merged[nino_cols].corrwith(merged['freeze_days'])

# Printing the correlations 
correlations

NINO1+2 Anom   -0.138668
NINO3 Anom     -0.162537
NINO4 Anom     -0.141433
NINO3.4 Anom   -0.141324
dtype: float64

The strongest correlation is with the NINO3 zone, with a correl coeffient of -0.16. There's a weak negative correlation between # of freeze days and NINO3. 

It seems like when there are La Nina conditions in the east pacific (negative NINO anom), there's greater risk of freeze days in Florida. Intuitively, this checks out.